In [1]:
# API request for one indicator

In [2]:
from fileUtils import fileDownload
from webScrape import transMonEE_web
from sdmx import sdmx_struc
import pandas as pd
import io

In [3]:
# TransMonEE list of countries from url web
url_transmonee = 'http://transmonee.org/countries/'
country_names = transMonEE_web.get_countries(url_transmonee)

In [4]:
# UNICEF’s REST API data endpoint
url_api = 'https://unicef.sdmxcloud.org/ws/public/sdmxapi/rest/data/'
# Dataflow information
d_flow = {'agency_id':'UNICEF', 'id_flow':'DM' ,'version':'1.0'}

In [5]:
# Request Dataflow structure, that provides important info:
# _ number of dimensions: required to properly point the API call
# _ country codes (not every Dataflow share the same country codes)

# address for dataflow structure request
api_address = url_api + ','.join(d_flow.values()) + '/all'
# parameters for dataflow structure request
api_params = {'format':'sdmx-json', 'detail':'structureOnly'}

# API request (dataflow structure)
d_flow_struc = fileDownload.api_request(api_address,api_params)

In [6]:
# get number of dimensions in SDMX dataflow
n_dim = sdmx_struc.get_sdmx_dim(d_flow_struc.json())
print(f"DataFlow {d_flow['id_flow']} has {n_dim} dimensions")

DataFlow DM has 6 dimensions


In [7]:
# get country codes dictionary from SDMX dataflow
country_codes = sdmx_struc.match_country_list(country_names,d_flow_struc.json())

In [8]:
# Indicator annual births, Helix: DM_BRTS
indicator_code = 'DM_BRTS'

In [9]:
# Join all countries required
country_call = '+'.join(country_codes.values())

# address for indicator request
# Assumes SDMX key position 1: countries, key position 2: indicator, remaining key positions empty
# SMDX key positions are separated by points

# make more clear in a function?
api_address = url_api + ','.join(d_flow.values()) + '/' + country_call + '.' + indicator_code + '.'*(n_dim-2)
# parameters for indicator request
api_params = {'format':'csv'}

In [10]:
# API request indicator
data_set = fileDownload.api_request(api_address,api_params)

In [11]:
# Read strings as pandas dataframe
data_set_df = pd.read_csv(io.StringIO(data_set.content.decode('utf-8')))
data_set_df.head()

,Geographic area,Indicator,Residence,Sex,Current age,Unit of measure,Unit multiplier,SOURCE_LINK,SERIES_FOOTNOTE,TIME_PERIOD,OBS_VALUE,Observation Status,Observation confidentaility,DATA_SOURCE,COVERAGE_TIME,Time interval at which the source data are collected,TIME_PERIOD_METHOD,OBS_FOOTNOTE
0,Albania,Number of births,Total,Total,Total,Persons,Thousands,NaN,NaN,2018,33.931,NaN,NaN,UNPD2019,NaN,NaN,NaN,NaN
1,Azerbaijan,Number of births,Total,Total,Total,Persons,Thousands,NaN,NaN,2018,166.969,NaN,NaN,UNPD2019,NaN,NaN,NaN,NaN
2,Armenia,Number of births,Total,Total,Total,Persons,Thousands,NaN,NaN,2018,41.197,NaN,NaN,UNPD2019,NaN,NaN,NaN,NaN
3,Bosnia and Herzegovina,Number of births,Total,Total,Total,Persons,Thousands,NaN,NaN,2018,27.133,NaN,NaN,UNPD2019,NaN,NaN,NaN,NaN
4,Bulgaria,Number of births,Total,Total,Total,Persons,Thousands,NaN,NaN,2018,62.823,NaN,NaN,UNPD2019,NaN,NaN,NaN,NaN


In [12]:
# write data_set to destination file
dest_path = './data_raw/'
dest_file = f"{dest_path}{indicator_code}.csv"
with open(dest_file, 'w+') as f:
    f.write(data_set.content.decode('utf-8'))